# My first LLM Project
Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary.

## Install required packages for LLM integration
Run this cell first - you'll see installation progress for each package. After running this, you'll be ready to import and use these packages in your LLM project!

In [ ]:
!pip3 install bs4 ollama requests

In [ ]:
import ollama
import requests

from bs4 import BeautifulSoup

In [ ]:
MODEL = "llama3"

In [ ]:
class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, features="html.parser")
        self.title = soup.title.string if soup.title else None

        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        self.text = soup.body.get_text(separator='\n', strip=True)
        

In [ ]:
website = Website("https://www.google.com")
print(website.title)
print(website.text)

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
def user_prompt_for(website_url):
    

In [ ]:
messages = [
    {"role": "user", "content": "Describe some of the business use cases of AI."},
]

In [ ]:
response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])